In [1]:
!pwd

/Users/ashish1610dhiman/data_projects/fall22_hw/RoboChef/notebooks/recommendation


In [26]:
import pandas as pd
from surprise import Dataset
from surprise import Reader

from surprise import accuracy, Dataset, SVD
from surprise.model_selection import train_test_split
from surprise import Dataset, SVD
from surprise.model_selection import cross_validate, GridSearchCV

import numpy as np

### Read the user-recipee dataset

In [5]:
raw_interactions = pd.read_csv("../../data/recommendation/RAW_interactions.csv")
raw_interactions.shape

(1132367, 5)

In [6]:
#cap ratings to 1 and 5
raw_interactions.columns

Index(['user_id', 'recipe_id', 'date', 'rating', 'review'], dtype='object')

In [8]:
raw_interactions["rating"].describe()

count    1.132367e+06
mean     4.411016e+00
std      1.264752e+00
min      0.000000e+00
25%      4.000000e+00
50%      5.000000e+00
75%      5.000000e+00
max      5.000000e+00
Name: rating, dtype: float64

In [19]:
reader = Reader(rating_scale=(1, 5))
interaction_model = Dataset.load_from_df(raw_interactions[["user_id", "recipe_id", "rating"]], reader)

In [14]:
0.7/0.85

0.8235294117647058

In [35]:
train_validation_set, test_set = train_test_split(interaction_model, test_size=0.15)

In [37]:
# We'll use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results
cross_validate(algo, interaction_model, measures=["RMSE", "MAE"], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2198  1.2212  1.2245  1.2164  1.2202  1.2204  0.0026  
MAE (testset)     0.7411  0.7400  0.7426  0.7377  0.7388  0.7400  0.0017  
Fit time          15.87   15.91   16.08   15.90   15.95   15.94   0.07    
Test time         2.69    2.70    1.47    1.45    2.81    2.22    0.63    


{'test_rmse': array([1.21981654, 1.22117497, 1.22453554, 1.21643379, 1.22024234]),
 'test_mae': array([0.74107881, 0.73995385, 0.74256786, 0.73765831, 0.73883264]),
 'fit_time': (15.865982055664062,
  15.914062976837158,
  16.077264070510864,
  15.897216081619263,
  15.948534965515137),
 'test_time': (2.694045066833496,
  2.7012999057769775,
  1.465512990951538,
  1.4543659687042236,
  2.806859016418457)}

In [33]:
param_grid = {"n_epochs": [5, 10], "lr_all": [0.002, 0.005], "reg_all": [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=5)

gs.fit(interaction_model)

# best RMSE score
print(gs.best_score["rmse"])

# combination of parameters that gave the best RMSE score
print(gs.best_params["rmse"])

1.217653341202198
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [34]:
print(gs.best_params["mae"])

{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
